## OPeNDAP Server/Client 

In [1]:
dap = 'https://hydro1.gesdisc.eosdis.nasa.gov/opendap/hyrax/FLDAS/'
resource = 'FLDAS_NOAH01_C_SA_MA.001/2013/FLDAS_NOAH01_C_SA_MA.ANOM201301.001.nc'
url = dap + resource

In [ ]:
...
username, _, password = netrc().hosts['urs.earthdata.nasa.gov']

In [ ]:
from pydap.client import ...
from pydap.cas.urs import ...

session = setup_session(username, password, check_url = url)
... = open_url(..., session = session)

In [ ]:
varname = ...
variable = dataset[varname]
variable.shape

In [ ]:
data = ...
dims = {k: v.data for k, v in var.maps.items()}
nodata = dataset.attributes['NC_GLOBAL']['missing_value']

## Data Alignment 

In [ ]:
...

data = np.flip(data, 1).astype('float32')
nodata = data.dtype.type(nodata)

In [ ]:
...

meta = {
    'driver': 'GTiff',
    'dtype': data.dtype,
    'count': ...,
    'height': ...,
    'width': ...,
    'nodata': nodata,
}
with raster(varname + '.tif', ..., **meta) as r:
    r.write(data[0, :, :], 1)

In [ ]:
from rasterio.plot import show

with raster(varname + '.tif') as r:
    show(r.read(1, masked = True))

In [ ]:
from rasterio.crs import ...
from rasterio.transform import ...

crs = CRS.from_epsg(...) # a guess!
attr = dataset.attributes['NC_GLOBAL']
transform = from_origin(
    dims['lon'][0].item(), # west
    dims['lat'][-1].item(), # north (flip!)
    attr['DX'], # xsize
    attr['DY']) # ysize

In [ ]:
...({
    'crs': crs,
    'transform': transform,
})
with raster(varname + '.tif', 'w', **meta) as r:
    r.write(data[0, :, :], 1)

In [ ]:
with raster(varname + '.tif') as r:
    show((r, 1))

In [ ]:
import geopandas as gpd
fig, ax = plt.subplots()

basin = ...(
    '/data/Aqueduct_river_basins_LIMPOPO')
with raster(varname + '.tif') as r:
    show((r, 1), ax = ax)
basin.plot(...,
    color='none', edgecolor = 'black')

## Subsetted Requests 

In [ ]:
from rasterio.mask import ...
from shapely.geometry import mapping

feature = [mapping(g) for g in basin['geometry']]
with raster(varname + '.tif') as r:
    ... = mask(r, feature)
    meta = r.meta.copy()

In [ ]:
with raster(varname + '_basin.tif', 'w', **meta) as r:
    r.write(masked)

In [ ]:
from rasterio.windows import ...

with raster(varname + '_basin.tif') as r:
    var = r.read(1, masked = True)

y, x = get_data_window(var)
basin = var.mask[slice(*y), slice(*x)]

In [ ]:
y = variable.shape[1] - y[1], variable.shape[1] - y[0]
var = variable[:, slice(*y), slice(*x)]
show(var)

## Loop though Time

In [ ]:
from pandas import Series
basin_ts = Series(index = [[],[]])

name = 'FLDAS_NOAH01_C_SA_MA'
resource = '{0}.001/{1}/{0}.ANOM{1}{2:02d}.001.nc'    

yr = 2016
mo = 0
while True:

    # connect to resource
    # for year and month
    url = dap + resource.format(name, yr, mo + 1)
    try:
        dataset = open_url(url,
            session = session)
    except:
        break
    
    # request data subset
    # and store masked mean
    variable = dataset[varname]
    var = variable[:, slice(*y), slice(*x)]
    data = np.flip(var.array.data, 1)
    data = np.ma.array(
        data,
        mask = basin)
    basin_ts[yr, mo + 1] = data.mean()
    
    # increment month and year
    mo = (mo + 1) % 12
    yr = yr + 1 if mo == 0 else yr